<a href="https://colab.research.google.com/github/FinCode-17/Options-Futures/blob/main/Binomial_American_Call_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt

def BS(S,K,T,r,sigma):
    d2 = (np.log(S/K)+(r-sigma**2/2)*T)/(sigma*np.sqrt(T))
    return S*ss.norm.cdf(d2+sigma*np.sqrt(T))-K*np.exp(-r*T)*ss.norm.cdf(d2)

def BSAmeriCallBound(S,K,T,r,sigma,d,TD):
    ShortCumDiv = BS(S,K,TD,r,sigma)
    LongExDiv = BS(S*(1-d),K,T,r,sigma)
    return np.maximum(ShortCumDiv,LongExDiv)

In [ ]:
def imat(n):
    B = np.empty((n+1,n+1,))
    B[:] = np.nan
    return B

def maketree(S,u,d,n):
    B = imat(n)
    B[0][0] = S
    for i in range(n):
        B[0][i+1] = B[0][i]*u
        for j in range(i+1):
            B[j+1][i+1] = B[j][i+1]*d/u
    return B

def CRRTree(S,sigma,T,n):

    u = np.exp(sigma*np.sqrt(T/n))
    return maketree(S,u,1/u,n)

In [ ]:
def RNP(PO,q,r):
    n = len(PO)
    M = imat(n)
    M[:,-1] = PO
    for i in range(n):
        M[0:n-i,n-1-i] = (q*M[0:n-i,n-i]+(1-q)*M[1:n+1-i,n-i])/(1+r)
    return M

def CRRCall(S,K,T,r,sigma,n):
    CRRT = CRRTree(S,sigma,T,n)
    u = CRRT[0,1]/CRRT[0,0]
    r1p = np.exp(r*T/n)-1
    PO = CRRT[:,-1] - K
    PO[PO < 0] = 0
    return RNP(PO,(1+r1p-1/u)/(u-1/u),r1p)



S = 100
K = 100
r = 0.05
sigma = 0.5
T = 1/4
n = 300

print(BS(S,K,T,r,sigma))
print(CRRCall(S,K,T,r,sigma,n)[0,0])